In [1]:
from threading import Thread #线程
from queue import Queue #队列

class ThreadedGenerator(object):#线程生成器

    def __init__(self, iterator,
                 sentinel=object(),#传进一个有效的对象
                 queue_maxsize=0,#队列最大默认值
                 daemon=False):#不以deamon形式运行
        self._iterator = iterator
        self._sentinel = sentinel
        self._queue = Queue(maxsize=queue_maxsize)
        self._thread = Thread(#运行一个新的线程
            name=repr(iterator),#迭代器，以repr格式初始操作
            target=self._run
        )
        self._thread.daemon = daemon
        self._started = False

    def __repr__(self):
        return 'ThreadedGenerator({!r})'.format(self._iterator)

    def _run(self):
        try:
            for value in self._iterator:
                if not self._started:
                    return
                self._queue.put(value)
        finally:
            self._queue.put(self._sentinel)

    def close(self):
        self._started = False
        try:
            while True:
                self._queue.get(timeout=30)
        except KeyboardInterrupt as e:
            raise e
        except:
            pass

    def __iter__(self):
        self._started = True
        self._thread.start()
        for value in iter(self._queue.get, self._sentinel):
            yield value#生成器函数，记住上一次的位置
        self._thread.join()
        self._started = False

    def __next__(self):
        if not self._started:
            self._started = True
            self._thread.start()
        value = self._queue.get(timeout=30)
        if value == self._sentinel:
            raise StopIteration()
        return value

def test():

    def gene():
        i = 0
        while True:
            yield i
            i += 1

    t = gene()
    test = ThreadedGenerator(t)

    for _ in range(10):
        print(next(test))

    test.close()

if __name__ == '__main__':
    test()


0
1
2
3
4
5
6
7
8
9
